This notebook generates the xon17 annotations, i.e. each gene is annotated as follows:
* 0 if the gene has no mutation
* 1 if the gene is mutated but has no deleterious mutations
* 1 + (x/17) if the mutation is deleterious according to `x` algorithms of Annovar. In reality first a sum over all 17 algorithms for each protein mutation is computed, followed by an average over the entire gene across all protein mutations.

In [1]:
import sys

sys.path.append("../../vae_zinb_reprn/")
sys.path.append("../src/")

In [2]:
import numpy as np
import pandas as pd

import datetime
import logging
import os
import time
from functools import cached_property

from sklearn.metrics import average_precision_score, ndcg_score, roc_auc_score

In [3]:
from data import (
    construct_anno_features,
    construct_raw_mutation_features,
    construct_raw_cnv_features,
    canonicalize_mutations,
    ALIAS_TO_CANONICAL_NAME_MAP,
    construct_anno_features_xon17
)

genes_324 = pd.read_csv("../../data/druid_1.3_data/gene2ind.txt", header=None)[
    0
].tolist()

In [4]:
%matplotlib inline

In [5]:
# Set default logging level
# Change to logging.INFO to see related output
logging.basicConfig(level=logging.WARNING, force=True)
pd.set_option("display.max_columns", 20)

In [6]:
df = pd.read_table(
    "../data/raw/CCLE.hg38.DepMap21Q3.annot.subset.cols.txt", low_memory=False,
)
print(df.shape)
df.head()

(755969, 61)


,input,DepMap-ID,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,LRT_score,LRT_converted_rankscore,...,fathmm.XF_coding_rankscore,fathmm.XF_coding_pred,integrated_fitCons_score,integrated_fitCons_rankscore,integrated_confidence_value,GERP.._NR,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons30way_mammalian,phastCons30way_mammalian_rankscore
0,VPS13D_S2041X,ACH-000001,.,.,.,.,.,.,0.000,0.843,...,0.289,N,0.722,0.854,0,5.77,1.000,0.716,1.000,0.863
1,ZSCAN20_V165G,ACH-000001,0.13,0.267,T,0.045,0.494,D,0.824,0.069,...,0.054,N,0.615,0.376,0,6.05,0.000,0.064,0.021,0.142
2,POU3F1_G93R,ACH-000001,0.494,0.079,T,0.228,0.253,T,0.480,0.122,...,0.203,N,0.598,0.345,0,3.49,1.000,0.716,0.999,0.704
3,MAST2_A739V,ACH-000001,.,.,.,0.214,0.264,T,.,.,...,0.000,N,0.707,0.731,0,4.83,0.012,0.187,0.318,0.243
4,NBPF9_K23T,ACH-000001,.,.,.,0.023,0.706,D,.,.,...,0.032,N,0.706,0.612,0,0.521,0.000,0.064,0.002,0.070


In [7]:
ccle_mutations_df = pd.read_csv("../data/raw/CCLE_mutations.csv", low_memory=False,)
print(ccle_mutations_df.shape)
ccle_mutations_df.head()

(1226336, 32)


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,34:221,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,9:0,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,silent,118:0,NaN,NaN,10:0,118:0,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,damaging,NaN,NaN,NaN,6:28,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,other non-conserving,28:62,NaN,NaN,NaN,27:61,NaN


In [8]:
# ccle_cnv_df = pd.read_csv(
#     "../data/raw/CCLE_cn_segments_annotated_21Q3.txt", delimiter="\t"
# )
# ccle_cnv_df.rename(columns={"DepMap-ID": "depmap_id"}, inplace=True)
# print(ccle_cnv_df.shape)
# ccle_cnv_df.head()

In [9]:
# ccle_cnv_pvt = pd.pivot_table(
#     ccle_cnv_df, "status", index=["depmap_id"], columns=["genesymbol"], aggfunc="first"
# )

# #
# # ccle_cnv_pvt = ccle_cnv_pvt.replace({'U':0, '+':1, '-':-1})

# # Binary
# ccle_cnv_pvt = ccle_cnv_pvt.replace({"U": 0, "+": 1, "-": 1})
# print(
#     "number of intersecting genes:",
#     np.intersect1d(ccle_cnv_pvt.columns.tolist(), genes_324).size,
# )

# missing_columns = np.setdiff1d(genes_324, ccle_cnv_pvt.columns.tolist())
# print(
#     "genes that are missing from depmap: ",
#     missing_columns,
# )

In [10]:
# ccle_cnv_pvt.columns = [col.upper() for col in ccle_cnv_pvt.columns]

In [11]:
# # mapping 'missing genes' to synonyms {ccle:324genes}
# # cnv_dict = {
# #     "GID4": "C17ORF39",
# #     "AMER1": "FAM123B",
# #     "KMT2A": "MLL",
# #     "KMT2D": "MLL2",
# #     "MYCL": "MYCL1",
# #     "MRE11": "MRE11A",
# #     "TENT5C": "FAM46C",
# #     "PRKN": "PARK2",
# #     "NSD2": "WHSC1",
# #     "NSD3": "WHSC1L1",
# # }
# # NOTE: use ALIAS_TO_CANONICAL_NAME_MAP instead of the above as it covers
# # lot more aliases than this list

# columns_to_rename = {
#     k: v for k, v in ALIAS_TO_CANONICAL_NAME_MAP.items() if v in missing_columns
# }
# ccle_cnv_pvt_renamed = ccle_cnv_pvt.rename(columns=columns_to_rename)

In [12]:
# print(
#     "genes that are missing from depmap: ",
#     np.setdiff1d(genes_324, ccle_cnv_pvt_renamed.columns.tolist()),
# )

In [13]:
# ccle_cnv_pvt_renamed["TERC"] = 0
# ccle_cnv_pvt_324 = ccle_cnv_pvt_renamed.loc[:, ccle_cnv_pvt_renamed.columns.isin(genes_324)].copy()

In [14]:
# ccle_cnv_pvt_324.shape

In [15]:
# for col in genes_324:
#     if len(ccle_cnv_pvt_324[col].shape) > 1:
#         print(col)

In [16]:
# duplicated_columns = [
#     "BRIP1",
#     "CHEK2",
#     "CXCR4",
#     "DDR2",
#     "ERCC4",
#     "FANCA",
#     "FBXW7",
#     "FGFR1",
#     "GNAQ",
#     "HNF1A",
#     "IRF4",
#     "KDM5A",
#     "MED12",
#     "MEN1",
#     "NFE2L2",
#     "NKX2-1",
#     "PTEN",
#     "SOX9",
#     "XPO1",
# ]

# {k:v for k,v in ALIAS_TO_CANONICAL_NAME_MAP.items() if ((v in duplicated_columns)  and (k in ccle_cnv_pvt.columns))}

In [17]:
# for col in ccle_cnv_pvt_324.columns:
#     ccle_cnv_pvt_324.loc[:, col] = ccle_cnv_pvt_324[col].astype(np.integer)

In [18]:
# ccle_cnv_pvt_324

## Sample validation

In [19]:
# ccle_cnv_df.genesymbol = ccle_cnv_df.genesymbol.replace(
#     list(columns_to_rename.keys()), list(columns_to_rename.values())
# )

In [20]:
# unique_depmap_ids = ccle_cnv_pvt_324.index.get_level_values(0)
# rand_patient_idx = np.random.randint(0, len(unique_depmap_ids))
# depmap_id = unique_depmap_ids[rand_patient_idx]
# temp_df = ccle_cnv_df[ccle_cnv_df.depmap_id == depmap_id]
# display(temp_df[(temp_df.genesymbol.isin(genes_324)) & temp_df.status.isin(["+", "-"])])

# ccle_cnv_pvt_324.columns[ccle_cnv_pvt_324.loc[depmap_id].values != 0]

In [21]:
# ccle_cnv_pvt_324.to_csv("../data/processed/ccle_cnv_binary.csv")

In [8]:
df_auc = pd.read_csv("../../data/druid_1.3_data/cell_drug_auc_final_1111.csv")
df_auc["depmap_id"] = df_auc["ARXSPAN_ID"].astype("string")
df_auc.drop("ARXSPAN_ID", axis=1, inplace=True)
df_auc.set_index(["depmap_id"], inplace=True)
df_auc.head()

,JW-7-24-1,KIN001-260,NSC-87877,GNE-317,NAVITOCLAX,PLX-4720,ERK5-IN-1,VX-11E,TGX-221,DASATINIB,...,BIX02189,ISPINESIB MESYLATE,KIN001-135,KIN001-206,KIN001-236,KIN001-266,LUMINESPIB,NUTLIN-3A,SGC0946,SL 0101-1
depmap_id,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.778432,0.951321,0.840287,-99999.000000,-99999.000000,0.936410,0.891908,0.402122,0.659600,0.835903,...,0.982272,0.733492,0.072277,0.983536,-99999.000000,0.651746,0.982084,0.893777,0.983861,0.933035
ACH-000002,0.788327,0.874760,0.760137,0.905754,0.930764,0.991068,0.278288,0.972905,0.558774,0.890106,...,0.899628,0.604228,0.046061,0.918265,0.865628,-99999.000000,0.985921,0.839175,0.988639,0.533204
ACH-000004,0.737630,0.938733,0.835441,0.808964,0.966133,0.983552,-99999.000000,0.993912,0.596027,0.960320,...,0.965245,0.782295,0.366912,0.844979,0.761323,-99999.000000,0.980945,0.946694,0.992605,0.761956
ACH-000006,0.176396,0.571569,0.581729,0.585938,0.958510,0.935574,0.600428,-99999.000000,0.596056,0.690723,...,0.834158,0.605360,0.124930,0.813384,0.782187,-99999.000000,0.971058,0.871277,-99999.000000,0.495020
ACH-000007,0.569751,0.755449,0.954924,0.939587,0.965638,0.932666,-99999.000000,0.956481,0.838692,0.704093,...,0.927519,0.812955,0.719780,0.815099,0.795736,0.860768,0.983317,0.888067,0.987564,0.787948


In [9]:
ccle_mutations_df

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,34:221,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,9:0,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,silent,118:0,NaN,NaN,10:0,118:0,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,damaging,NaN,NaN,NaN,6:28,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,other non-conserving,28:62,NaN,NaN,NaN,27:61,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226331,SLC39A4,55630,37,8,145641991,145641991,+,Silent,SNP,C,...,False,0.0,0.000017,silent,52:23,NaN,NaN,NaN,NaN,NaN
1226332,TAL2,6887,37,9,108424778,108424778,+,Start_Codon_SNP,SNP,A,...,False,0.0,NaN,damaging,27:0,NaN,NaN,NaN,NaN,NaN
1226333,TRO,7216,37,X,54955098,54955098,+,Silent,SNP,C,...,False,0.0,NaN,silent,5:16,NaN,NaN,NaN,NaN,NaN
1226334,USP51,158880,37,X,55514703,55514703,+,Missense_Mutation,SNP,G,...,False,0.0,NaN,other non-conserving,23:0,NaN,NaN,NaN,NaN,NaN


In [10]:
# mutations_needed = ["damaging", "other non-conserving"]

ccle_mutations_df = ccle_mutations_df[
    ["Hugo_Symbol", "DepMap_ID", "Protein_Change", "Variant_annotation"]
].copy()
print(ccle_mutations_df.shape)
ccle_mutations_filtered = ccle_mutations_df#[
#     ccle_mutations_df["Variant_annotation"].isin(mutations_needed)
# ].copy()
print(ccle_mutations_filtered.shape)
ccle_mutations_filtered.head()

(1226336, 4)
(1226336, 4)


,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation
0,VPS13D,ACH-000001,p.S2041*,damaging
1,AADACL4,ACH-000001,p.WRDAI263del,other non-conserving
2,IFNLR1,ACH-000001,p.G337G,silent
3,TMEM57,ACH-000001,p.K264fs,damaging
4,ZSCAN20,ACH-000001,p.V165G,other non-conserving


In [26]:
# Not filtering out any cell lines or mutations at this point

# ccle_mutations_filtered.rename(columns={"DepMap_ID": "depmap_id"}, inplace=True)
# ccle_mutations_filtered = ccle_mutations_filtered[
#     ccle_mutations_filtered.depmap_id.isin(df_auc.index.get_level_values(0))
# ]
# ccle_mutations_filtered

In [11]:
ccle_mutations_filtered.Hugo_Symbol = ccle_mutations_filtered.Hugo_Symbol.str.upper()

In [12]:
print(
    "number of intersecting genes:",
    np.intersect1d(ccle_mutations_filtered.Hugo_Symbol.unique(), genes_324).size,
)

missing_genes_in_mutations = np.setdiff1d(genes_324, ccle_mutations_filtered.Hugo_Symbol.unique())
print(
    "genes that are missing from depmap: ",
    missing_genes_in_mutations,
)

number of intersecting genes: 317
genes that are missing from depmap:  ['C17ORF39' 'EMSY' 'FAM123B' 'MLL' 'MLL2' 'MYCL1' 'TERC']


In [13]:
# mut_dict = {
#     "GID4": "C17ORF39",
#     "C11orf30": "EMSY",
#     "AMER1": "FAM123B",
#     "KMT2A": "MLL",
#     "KMT2D": "MLL2",
#     "MYCL": "MYCL1",
# }
rows_to_rename_in_mutations = {
    k: v for k, v in ALIAS_TO_CANONICAL_NAME_MAP.items() if v in missing_genes_in_mutations
}
ccle_mutations_filtered.Hugo_Symbol.replace(
    list(rows_to_rename_in_mutations.keys()), list(rows_to_rename_in_mutations.values()), inplace=True
)

In [14]:
print(
    "number of intersecting genes:",
    np.intersect1d(ccle_mutations_filtered.Hugo_Symbol.unique(), genes_324).size,
)

print(
    "genes that are missing from depmap: ",
    np.setdiff1d(genes_324, ccle_mutations_filtered.Hugo_Symbol.unique()),
)

number of intersecting genes: 323
genes that are missing from depmap:  ['TERC']


In [15]:
# Filtering out and using only the F1 genes
ccle_mutations_filtered = ccle_mutations_filtered[
    ccle_mutations_filtered.Hugo_Symbol.isin(genes_324)
]
# Not filtering out NaN protein mutations
# ccle_mutations_filtered = ccle_mutations_filtered[
#     ~ccle_mutations_filtered.Protein_Change.isna()
# ]
ccle_mutations_filtered["mutation"] = (
    ccle_mutations_filtered["Hugo_Symbol"]
    + " "
    + ccle_mutations_filtered["Protein_Change"].apply(lambda val: val[2:] if val is not np.NaN else "")
)
ccle_mutations_filtered["mutation"] = ccle_mutations_filtered["mutation"].str.strip(" ")
ccle_mutations_filtered

<ipython-input-15-5bd49e41b828>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccle_mutations_filtered["mutation"] = (
<ipython-input-15-5bd49e41b828>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccle_mutations_filtered["mutation"] = ccle_mutations_filtered["mutation"].str.strip(" ")


,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation
89,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs
95,TP53,ACH-000001,NaN,other non-conserving,TP53
108,CD79B,ACH-000001,p.L21L,silent,CD79B L21L
109,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L
123,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C
...,...,...,...,...,...
1226149,NTRK1,ACH-002512,p.D270D,silent,NTRK1 D270D
1226178,MLL2,ACH-002512,p.V5399V,silent,MLL2 V5399V
1226212,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y
1226223,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A


In [31]:
# annotated_df = pd.DataFrame(columns=['input', 'sift_pred', 'sift4g_pred', 'lrt_pred', 'mutationtaster_pred',
#        'mutationassessor_pred', 'fathmm_pred', 'provean_pred', 'metasvm_pred',
#        'm_cap_pred', 'primateai_pred', 'deogen2_pred', 'bayesdel_addaf_pred',
#        'bayesdel_noaf_pred', 'clinpred_pred', 'list_s2_pred',
#        'fathmm_mkl_coding_pred', 'fathmm_xf_coding_pred', 'gene'])
# for depmap_id in ccle_mutations_filtered.depmap_id.unique():
#     filtered_df = ccle_mutations_filtered[
#         ccle_mutations_filtered.depmap_id == depmap_id
#     ]
#     point_mutations = list(filtered_df.mutation.values)
#     _, annot_df = construct_anno_features_xon17_gpd(depmap_id, point_mutations, False)
#     annotated_df = pd.concat([annotated_df, annot_df])

ERROR:root:Encountered error while processing mutation TSC2 F297fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 K1074E - No columns to parse from file
ERROR:root:Encountered error while processing mutation CEBPA 186_189PPPP>P - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A R29* - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5C E1101del - No columns to parse from file
ERROR:root:Encountered error while processing mutation ASXL1 L1393fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRAF E204L - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 D560H - No columns to parse from file
ERROR:root:Encountered error while processing mutation HNF1A S513L - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 V218del - No columns to parse from

ERROR:root:Encountered error while processing mutation TP53 H233del - No columns to parse from file
ERROR:root:Encountered error while processing mutation PPARG K156fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 S409G - No columns to parse from file
ERROR:root:Encountered error while processing mutation FGFR1 G103C - No columns to parse from file
ERROR:root:Encountered error while processing mutation MDM4 D375C - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A1598V - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRCA2 AK1689fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MST1R V323fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 S637L - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 LF1246fs - No columns to parse from file
ER

ERROR:root:Encountered error while processing mutation C17ORF39 V148M - No columns to parse from file
ERROR:root:Encountered error while processing mutation SPEN E327fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3R1 90_91insN - No columns to parse from file
ERROR:root:Encountered error while processing mutation RICTOR H1109fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P972fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S717Y - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B G1031D - No columns to parse from file
ERROR:root:Encountered error while processing mutation REL I543fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 P191del - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN CS211fs - No columns to parse from f

ERROR:root:Encountered error while processing mutation APC E1554fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A Y44fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation IRS2 NNN26del - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 CNSSC238fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 R944Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 E4662K - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A I154fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A V153del - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S5385L - No columns to parse from file
ERROR:root:Encountered error while processing mutation TSC1 L557fs - No columns to parse from fil

ERROR:root:Encountered error while processing mutation STK11 L55fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation SNCAIP L918fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ATRX L359fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ATM K1903fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q4228E - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 T2131P - No columns to parse from file
ERROR:root:Encountered error while processing mutation SMAD4 P514fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 R420fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q2736* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 T2131P - No columns to parse from file
ERRO

...

ERROR:root:Encountered error while processing mutation PIK3C2G W126fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 V3376M - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A3292T - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Y2907fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 E922fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation HNF1A G522fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FLT1 M609fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation IRS2 P1091fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation AKT1 G6D - No columns to parse from file
ERROR:root:Encountered error while processing mutation AKT1 K2N - No columns to parse from file
ERROR:root

ERROR:root:Encountered error while processing mutation MLL2 P1931H - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 E1517D - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P1152H - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRCA2 C1200fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRCA2 Q1782fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ETV4 F18fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ETV4 PYT15fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation RNF43 G659fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation SOX9 S414fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MSH6 G289fs - No columns to parse from file
ERROR

ERROR:root:Encountered error while processing mutation MLL2 R4966Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R4960* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R4282* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 M4148V - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R3321* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R2915Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P2275S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S2040N - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R1292C - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R625H - No columns to parse from file
ERROR:root:

ERROR:root:Encountered error while processing mutation DDR1 Y185fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation DAXX T249fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation SGK1 K367del - No columns to parse from file
ERROR:root:Encountered error while processing mutation TNFAIP3 DP756fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation XRCC2 L117fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCORL1 S1679fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q4079* - No columns to parse from file
ERROR:root:Encountered error while processing mutation NTRK1 AVGL421del - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S4927R - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 Q705* - No columns to parse from

ERROR:root:Encountered error while processing mutation EP300 K1469fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MAP3K1 S1077fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation STAG2 F120fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 T2131P - No columns to parse from file
ERROR:root:Encountered error while processing mutation ATM F1068fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FLCN 379_380RD>N - No columns to parse from file
ERROR:root:Encountered error while processing mutation ALK C928fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MYCL1 R261M - No columns to parse from file
ERROR:root:Encountered error while processing mutation PARP3 EAL241del - No columns to parse from file
ERROR:root:Encountered error while processing mutation SPEN P3120del - No columns to parse from

In [32]:
# annotated_df.to_csv("../data/processed/anno_features_per_mutation_ccle.csv")

In [31]:
# raw_mutations_combined = []
# for depmap_id in ccle_mutations_filtered.depmap_id.unique():
#     filtered_df = ccle_mutations_filtered[
#         ccle_mutations_filtered.depmap_id == depmap_id
#     ]
#     point_mutations = list(filtered_df.mutation.values)
#     raw_mutations_combined.append(
#         construct_raw_mutation_features(depmap_id, point_mutations)
#     )
# raw_mutations_combined_df = pd.concat(raw_mutations_combined)
# raw_mutations_combined_df.index.name = "depmap_id"
# raw_mutations_combined_df

In [32]:
# rand_patient_idx = np.random.randint(0, 691)
# display(
#     ccle_mutations_filtered[
#         ccle_mutations_filtered.depmap_id
#         == raw_mutations_combined_df.iloc[rand_patient_idx].name
#     ]
# )

# raw_mutations_combined_df.columns[
#     raw_mutations_combined_df.iloc[rand_patient_idx].values != 0
# ]

In [33]:
# raw_mutations_combined_df.to_csv("../data/processed/ccle_raw_mutation.csv")

## Variant Annotations with Transvar/Annovar

In [48]:
# variant annotation on these point mutations
import sys
sys.path.append("../src/")
from data import (construct_anno_features, 
                  construct_raw_cnv_features, 
                  construct_raw_mutation_features, 
                  canonicalize_mutations,
                  ALIAS_TO_CANONICAL_NAME_MAP, 
                  _is_valid_point_mutations,
                  get_annotation_features,
                  preprocess_annotation_features,
                  MICEData
                 )

In [49]:
import logging
import re
import subprocess
import tempfile
import itertools
import json

In [50]:
REQUIRED_ANNOTATION_COLUMNS = [
    "SIFT_score",
    "SIFT_converted_rankscore",
    "SIFT_pred",
    "SIFT4G_score",
    "SIFT4G_converted_rankscore",
    "SIFT4G_pred",
    "LRT_score",
    "LRT_converted_rankscore",
    "LRT_pred",
    "MutationTaster_score",
    "MutationTaster_converted_rankscore",
    "MutationTaster_pred",
    "MutationAssessor_score",
    "MutationAssessor_rankscore",
    "MutationAssessor_pred",
    "FATHMM_score",
    "FATHMM_converted_rankscore",
    "FATHMM_pred",
    "PROVEAN_score",
    "PROVEAN_converted_rankscore",
    "PROVEAN_pred",
    "MetaSVM_pred",
    "M-CAP_score",
    "M-CAP_rankscore",
    "M-CAP_pred",
    "MVP_score",
    "MVP_rankscore",
    "MPC_score",
    "MPC_rankscore",
    "PrimateAI_score",
    "PrimateAI_rankscore",
    "PrimateAI_pred",
    "DEOGEN2_score",
    "DEOGEN2_rankscore",
    "DEOGEN2_pred",
    "BayesDel_addAF_score",
    "BayesDel_addAF_pred",
    "BayesDel_noAF_score",
    "BayesDel_noAF_rankscore",
    "BayesDel_noAF_pred",
    "ClinPred_score",
    "ClinPred_rankscore",
    "ClinPred_pred",
    "LIST-S2_score",
    "LIST-S2_rankscore",
    "LIST-S2_pred",
    "DANN_score",
    "DANN_rankscore",
    "fathmm-MKL_coding_score",
    "fathmm-MKL_coding_rankscore",
    "fathmm-MKL_coding_pred",
    "fathmm-XF_coding_score",
    "fathmm-XF_coding_rankscore",
    "fathmm-XF_coding_pred",
    "Eigen-raw_coding",
    "Eigen-raw_coding_rankscore",
    "Eigen-PC-raw_coding",
    "Eigen-PC-raw_coding_rankscore",
]
CATEGORICAL_COLUMNS = [
    "sift_pred",
    "sift4g_pred",
    "lrt_pred",
    "mutationtaster_pred",
    "mutationassessor_pred",
    "fathmm_pred",
    "provean_pred",
    "metasvm_pred",
    "m_cap_pred",
    "primateai_pred",
    "deogen2_pred",
    "bayesdel_addaf_pred",
    "bayesdel_noaf_pred",
    "clinpred_pred",
    "list_s2_pred",
    "fathmm_mkl_coding_pred",
    "fathmm_xf_coding_pred",
]

# The thresholds used in PREDICTOR_LAMBDA_MAP are taken from the corresponding
# technique's published paper/web page
PREDICTOR_LAMBDA_MAP = {
    "sift_pred": ("sift_score", lambda v: "D" if v <= 0.05 else "T"),
    "sift4g_pred": ("sift4g_score", lambda v: "D" if v <= 0.05 else "T"),
    "lrt_pred": ("lrt_score", lambda v: "D" if v <= 0.001 else "U"),
    "mutationtaster_pred": (
        "mutationtaster_score",
        lambda v: None,
    ),  # Threshold is not available and couldn't be derived from available values as well
    "mutationassessor_pred": (
        "mutationassessor_score",
        lambda v: "H" if v >= 3.5 else ("M" if v >= 1.94 else "L"),
    ),
    "fathmm_pred": ("fathmm_score", lambda v: "D" if v < 1.5 else "T"),
    "provean_pred": ("provean_score", lambda v: "D" if v <= 2.282 else "N"),
    "metasvm_pred": (
        "metasvm_pred",
        lambda v: None,
    ),  # No corresponding numeric score available for this method
    "m_cap_pred": ("m_cap_score", lambda v: "D" if v >= 0.025 else "T"),
    "primateai_pred": ("primateai_score", lambda v: "D" if v >= 0.803 else "T"),
    "deogen2_pred": ("deogen2_score", lambda v: "D" if v >= 0.45 else "T"),
    "bayesdel_addaf_pred": (
        "bayesdel_addaf_score",
        lambda v: "D" if v >= 0.0692 else "T",
    ),
    "bayesdel_noaf_pred": (
        "bayesdel_noaf_score",
        lambda v: "D" if v >= -0.0570 else "T",
    ),
    "clinpred_pred": ("clinpred_score", lambda v: "D" if v >= 0.5 else "T"),
    "list_s2_pred": ("list_s2_score", lambda v: "D" if v >= 0.85 else "T"),
    "fathmm_mkl_coding_pred": (
        "fathmm_mkl_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
    "fathmm_xf_coding_pred": (
        "fathmm_xf_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
}

DELETERIOUS_VALUES = ["D", "A", "H", "M"]

CNV_PATTERN = r"loss|amplification"

GENES_324 = pd.read_csv("../../data/druid_1.3_data/gene2ind.txt", header=None)[
    0
].tolist()

ANNOTATION_SCRIPT_PATH = "../script/goAAtoGv2.sh"
SPECIAL_CASES = r"rearrangement|truncation|fs|del|ins"

In [76]:
# variant annotation on these point mutations
import sys
sys.path.append("../src/")
from data import (construct_anno_features, 
                  construct_raw_cnv_features, 
                  construct_raw_mutation_features, 
                  canonicalize_mutations,
                  ALIAS_TO_CANONICAL_NAME_MAP, 
                  _is_valid_point_mutations,
                  get_annotation_features,
                  preprocess_annotation_features,
                  MICEData
                 )

import logging
import re
import subprocess
import tempfile
import itertools
import json

REQUIRED_ANNOTATION_COLUMNS = [
    "SIFT_score",
    "SIFT_converted_rankscore",
    "SIFT_pred",
    "SIFT4G_score",
    "SIFT4G_converted_rankscore",
    "SIFT4G_pred",
    "LRT_score",
    "LRT_converted_rankscore",
    "LRT_pred",
    "MutationTaster_score",
    "MutationTaster_converted_rankscore",
    "MutationTaster_pred",
    "MutationAssessor_score",
    "MutationAssessor_rankscore",
    "MutationAssessor_pred",
    "FATHMM_score",
    "FATHMM_converted_rankscore",
    "FATHMM_pred",
    "PROVEAN_score",
    "PROVEAN_converted_rankscore",
    "PROVEAN_pred",
    "MetaSVM_pred",
    "M-CAP_score",
    "M-CAP_rankscore",
    "M-CAP_pred",
    "MVP_score",
    "MVP_rankscore",
    "MPC_score",
    "MPC_rankscore",
    "PrimateAI_score",
    "PrimateAI_rankscore",
    "PrimateAI_pred",
    "DEOGEN2_score",
    "DEOGEN2_rankscore",
    "DEOGEN2_pred",
    "BayesDel_addAF_score",
    "BayesDel_addAF_pred",
    "BayesDel_noAF_score",
    "BayesDel_noAF_rankscore",
    "BayesDel_noAF_pred",
    "ClinPred_score",
    "ClinPred_rankscore",
    "ClinPred_pred",
    "LIST-S2_score",
    "LIST-S2_rankscore",
    "LIST-S2_pred",
    "DANN_score",
    "DANN_rankscore",
    "fathmm-MKL_coding_score",
    "fathmm-MKL_coding_rankscore",
    "fathmm-MKL_coding_pred",
    "fathmm-XF_coding_score",
    "fathmm-XF_coding_rankscore",
    "fathmm-XF_coding_pred",
    "Eigen-raw_coding",
    "Eigen-raw_coding_rankscore",
    "Eigen-PC-raw_coding",
    "Eigen-PC-raw_coding_rankscore",
]
CATEGORICAL_COLUMNS = [
    "sift_pred",
    "sift4g_pred",
    "lrt_pred",
    "mutationtaster_pred",
    "mutationassessor_pred",
    "fathmm_pred",
    "provean_pred",
    "metasvm_pred",
    "m_cap_pred",
    "primateai_pred",
    "deogen2_pred",
    "bayesdel_addaf_pred",
    "bayesdel_noaf_pred",
    "clinpred_pred",
    "list_s2_pred",
    "fathmm_mkl_coding_pred",
    "fathmm_xf_coding_pred",
]

# The thresholds used in PREDICTOR_LAMBDA_MAP are taken from the corresponding
# technique's published paper/web page
PREDICTOR_LAMBDA_MAP = {
    "sift_pred": ("sift_score", lambda v: "D" if v <= 0.05 else "T"),
    "sift4g_pred": ("sift4g_score", lambda v: "D" if v <= 0.05 else "T"),
    "lrt_pred": ("lrt_score", lambda v: "D" if v <= 0.001 else "U"),
    "mutationtaster_pred": (
        "mutationtaster_score",
        lambda v: None,
    ),  # Threshold is not available and couldn't be derived from available values as well
    "mutationassessor_pred": (
        "mutationassessor_score",
        lambda v: "H" if v >= 3.5 else ("M" if v >= 1.94 else "L"),
    ),
    "fathmm_pred": ("fathmm_score", lambda v: "D" if v < 1.5 else "T"),
    "provean_pred": ("provean_score", lambda v: "D" if v <= 2.282 else "N"),
    "metasvm_pred": (
        "metasvm_pred",
        lambda v: None,
    ),  # No corresponding numeric score available for this method
    "m_cap_pred": ("m_cap_score", lambda v: "D" if v >= 0.025 else "T"),
    "primateai_pred": ("primateai_score", lambda v: "D" if v >= 0.803 else "T"),
    "deogen2_pred": ("deogen2_score", lambda v: "D" if v >= 0.45 else "T"),
    "bayesdel_addaf_pred": (
        "bayesdel_addaf_score",
        lambda v: "D" if v >= 0.0692 else "T",
    ),
    "bayesdel_noaf_pred": (
        "bayesdel_noaf_score",
        lambda v: "D" if v >= -0.0570 else "T",
    ),
    "clinpred_pred": ("clinpred_score", lambda v: "D" if v >= 0.5 else "T"),
    "list_s2_pred": ("list_s2_score", lambda v: "D" if v >= 0.85 else "T"),
    "fathmm_mkl_coding_pred": (
        "fathmm_mkl_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
    "fathmm_xf_coding_pred": (
        "fathmm_xf_coding_score",
        lambda v: "D" if v >= 0.5 else "N",
    ),
}

DELETERIOUS_VALUES = ["D", "A", "H", "M"]

CNV_PATTERN = r"loss|amplification"

GENES_324 = pd.read_csv("../../data/druid_1.3_data/gene2ind.txt", header=None)[
    0
].tolist()

ANNOTATION_SCRIPT_PATH = "../script/goAAtoGv2.sh"
SPECIAL_CASES = r"rearrangement|truncation|fs|del|ins"

In [109]:
def construct_anno_features_xon17_gpd(patient_id, patient_mutations, agg_features=False):
    """
    TODO: Add support for other agg functions (mean, OR, etc) - as of 202209, only
    sum is supported
    Here, the aggregation is done as an average over all variants over all 17 algorithms.
    """
    if agg_features:
        logging.warn(
            """
        Received agg_features=True -> As of now, construct_anno_features only supports sum aggregation.
        Please ensure that the agg used in dataset definition is sum - if it is not sum, please pass
        agg_features=False and perform agg in dataset definition
        """
        )

#     if not _is_valid_point_mutations(patient_mutations):
#         return None

    anno_features_combined_imputed_df = pd.read_csv(
        "../data/processed/anno_features_combined_imputed.csv"
    )
    logging.info(anno_features_combined_imputed_df.shape)
    anno_features_combined_imputed_df.set_index(["input"], inplace=True)
    anno_features_combined_imputed_df.head()

    canonical_mutations = canonicalize_mutations(patient_mutations)

    mutations_with_missing_annotations = []
    available_mutations = []
    for mutation in canonical_mutations:
        if mutation in anno_features_combined_imputed_df.index:
            available_mutations.append(mutation)
        elif not re.search(CNV_PATTERN, mutation, re.IGNORECASE):
            mutations_with_missing_annotations.append(mutation)

    if available_mutations:
        patient_anno_features = anno_features_combined_imputed_df.loc[
            available_mutations
        ]
        patient_anno_features = patient_anno_features[CATEGORICAL_COLUMNS].copy()
    else:
        patient_anno_features = None

    if len(mutations_with_missing_annotations) != 0:
        logging.info(
            f"Found mutations with missing annotations - {mutations_with_missing_annotations}"
        )
        missing_annotations = get_annotation_features(
            mutations_with_missing_annotations
        )
        if missing_annotations is not None:
            missing_annotations = missing_annotations[
                REQUIRED_ANNOTATION_COLUMNS
            ].copy()
            missing_annotations.reset_index(inplace=True)
            missing_annotations = preprocess_annotation_features(missing_annotations)
            missing_annotations = missing_annotations[~missing_annotations.duplicated()]
            missing_annotations.reset_index(drop=True, inplace=True)
            missing_annotations.set_index("input", inplace=True)

            numeric_columns = list(
                column
                for column in missing_annotations.columns
                if pd.api.types.is_numeric_dtype(missing_annotations[column])
            )
            # Prepare mask by identifying rows that have all na values for numeric_columns
            na_mask = None
            for col in numeric_columns:
                if type(na_mask) == pd.Series:
                    na_mask = na_mask & missing_annotations[col].isna()
                else:
                    na_mask = missing_annotations[col].isna()

            missing_annotations = missing_annotations[~na_mask]
            numeric_df = missing_annotations[numeric_columns].copy()
            logging.info(numeric_df.shape)
            numeric_df.head()
            numeric_df = pd.concat(
                [numeric_df, anno_features_combined_imputed_df[numeric_columns]],
            )

            categorical_columns = [
                column
                for column in missing_annotations.columns
                if column not in numeric_columns
            ]
            categorical_missing_annotations = missing_annotations[
                categorical_columns
            ].copy()
            logging.info(categorical_missing_annotations.shape)
            categorical_missing_annotations.head()

            imp = MICEData(numeric_df)
            # Impute missing values in numeric columns - Expensive!!
            imp.update_all()
            imputed_df = imp.data
            assert numeric_df.shape == imputed_df.shape
            imputed_df.index = numeric_df.index
            imputed_df = imputed_df[
                imputed_df.index.isin(mutations_with_missing_annotations)
            ].copy()
            numeric_imputed_df = pd.concat(
                [categorical_missing_annotations, imputed_df,], axis=1,
            )
            logging.info(numeric_imputed_df.shape)
            for column in CATEGORICAL_COLUMNS:
                logging.info(
                    column,
                    numeric_imputed_df[column].unique(),
                    len(numeric_imputed_df[numeric_imputed_df[column].isna()]),
                )
                col_na_mask = numeric_imputed_df[column].isna()
                numeric_imputed_df.loc[col_na_mask, column] = numeric_imputed_df[
                    col_na_mask
                ][PREDICTOR_LAMBDA_MAP[column][0]].apply(
                    PREDICTOR_LAMBDA_MAP[column][1]
                )

                # logging.info(
                #     column,
                #     numeric_imputed_df[column].unique(),
                #     len(numeric_imputed_df[numeric_imputed_df[column].isna()]),
                # )

            numeric_imputed_df = numeric_imputed_df.dropna()
            logging.info(numeric_imputed_df.shape)
            missing_anno_features_df = numeric_imputed_df[CATEGORICAL_COLUMNS].copy()

            patient_anno_features = pd.concat(
                [missing_anno_features_df, patient_anno_features]
            )
            
    if patient_anno_features is not None:
        for col in patient_anno_features.columns:
            patient_anno_features[col] = patient_anno_features[col].apply(
                lambda v: 1 if v in DELETERIOUS_VALUES else 0
            )

        patient_anno_features.reset_index(inplace=True)
        patient_anno_features["gene"] = patient_anno_features.input.apply(
            lambda gene_mut: gene_mut.split(" ")[0]
        )
    return patient_anno_features

In [36]:
# annotated_df = pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv", index_col = 0)
# annotated_df

,input,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,NOTCH1 C844G,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,NOTCH1
1,NOTCH3 R1981C,1,1,0,1,0,0,1,0,1,1,0,0,1,1,1,1,1,NOTCH3
2,PPP2R1A E100K,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,PPP2R1A
0,NRAS Q61L,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,NRAS
1,RPTOR D654N,1,1,1,1,0,0,1,0,1,0,1,0,0,1,1,1,1,RPTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,PPP2R2A E206A,0,0,0,1,0,0,1,0,0,0,0,1,1,1,0,1,1,PPP2R2A
5,NRAS Q61L,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,NRAS
6,FGF14 V2L,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,FGF14
7,IDH1 I99M,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,IDH1


In [38]:
# len(set(ccle_mutations_filtered["mutation"]) - set(annotated_df["input"]))

20419

In [39]:
# set(ccle_mutations_filtered["mutation"]) - set(annotated_df["input"])

{'EPHB1 T403T',
 'AXIN1 T374M',
 'MST1R D1208N',
 'MLL2 G1778G',
 'BCOR L1550L',
 'ATRX V1557G',
 'SDHA T126T',
 'CARD11 G126C',
 'FGFR2 S702S',
 'TP53 T211A',
 'MLL K1751*',
 'CASP8 Q184*',
 'SMO L522L',
 'FGFR2 M585R',
 'FAM123B G470G',
 'MLL2 Q3158*',
 'CDH1 Q610L',
 'PIK3CA L241L',
 'DOT1L L578L',
 'ARID1A G1976D',
 'EZH2 W504R',
 'RET R417C',
 'KEL I278V',
 'ARID1A S202S',
 'RPTOR C1168fs',
 'ZNF217 A934A',
 'MAPK1 V51V',
 'PRDM1 N136S',
 'AKT1 Q59*',
 'MLL R130W',
 'RAD52 G21G',
 'GATA3 T441T',
 'MLL2 D5519N',
 'PTPRO H9H',
 'MEF2B R219Q',
 'NOTCH1 P2514fs',
 'MLL2 T245S',
 'MLL2 V160M',
 'IGF1R A1346A',
 'FGF23 R23I',
 'DOT1L Y501C',
 'KEAP1 119_120GM>V',
 'ALOX12B A443A',
 'SF3B1 M446I',
 'PPARG K185fs',
 'FLT1 A289D',
 'ROS1 N2112K',
 'SGK1 I42F',
 'MLL A1560T',
 'MPL V124V',
 'KDM5A S686S',
 'ATR S601F',
 'GRM3 D542N',
 'IKBKE L14L',
 'MST1R L1356V',
 'EPHB1 L238H',
 'PIK3C2B P117S',
 'MYC L181L',
 'RPTOR L447L',
 'NOTCH1 L1559L',
 'KDM5A G1200fs',
 'FGFR3 H284R',
 'EGFR R703

In [105]:
import dask

from tqdm import tqdm
from dask.distributed import Client


client = Client()
client.cluster.scale(10)

/home/ajayago/anaconda3/envs/base2/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44633 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33053
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:44633/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35573'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40599'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37791'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34133'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42681'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42879'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36451'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44643'
INFO:distribu

In [67]:
%load_ext jupyterlab_notify

In [70]:
%%notify

# futures = []
anno_features_combined = pd.DataFrame()
for depmap_id in ccle_mutations_filtered.DepMap_ID.unique():
    filtered_df = ccle_mutations_filtered[
        ccle_mutations_filtered.DepMap_ID == depmap_id
    ]
    point_mutations = list(filtered_df.mutation.values)
    res = construct_anno_features_xon17_gpd(depmap_id, point_mutations, False)
    anno_features_combined = pd.concat([anno_features_combined, res], ignore_index = False)
#     future = client.submit(construct_anno_features_xon17_gpd, depmap_id, point_mutations, False)
#     futures.append(future)

# anno_features_combined = client.gather(futures, errors="skip")
# client.shutdown()

Point mutation TP53 is invalid
Point mutation PIK3R1 is invalid


ERROR:root:Encountered error while processing mutation CEBPA 186_189PPPP>P - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A R29* - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5C E1101del - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B G820G - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 D560H - No columns to parse from file
ERROR:root:Encountered error while processing mutation ASXL1 L1393fs - No columns to parse from file


Point mutation RAD54L is invalid


ERROR:root:Encountered error while processing mutation WHSC1 D560H - No columns to parse from file


Point mutation SMARCB1 is invalid


...

ERROR:root:Encountered error while processing mutation ABL1 R764W - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN LK182fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R5154Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 APAL1109fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 LPQVSPRRTG225fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NFKBIA VSLLLKCG233fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation SOCS1 *212S - No columns to parse from file
ERROR:root:Encountered error while processing mutation TET2 GW563fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation APC C344fs - No columns to parse from file


Point mutation TYRO3 is invalid


ERROR:root:Encountered error while processing mutation MLL2 S3351S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S1294S - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 F880F - No columns to parse from file
ERROR:root:Encountered error while processing mutation TET2 L1332H - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B G470V - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM46C G323G - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 K5362E - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 G436G - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 S33fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ALK C928fs - No columns to parse from file
ERROR:roo

Point mutation POLD1 is invalid
Point mutation MSH6 is invalid


ERROR:root:Encountered error while processing mutation MLL2 R1835H - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 V5399V - No columns to parse from file


In [71]:
anno_features_combined

,input,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,NRAS Q61L,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,NRAS
1,RPTOR D654N,1,1,1,1,0,0,1,0,1,0,1,0,0,1,1,1,1,RPTOR
0,CTNNB1 G245A,1,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1,CTNNB1
1,JAK1 I375V,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,JAK1
2,PIK3C2B R751W,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,PIK3C2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,BRCA1 H888Y,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,BRCA1
1,SMARCA4 A400A,1,1,1,1,1,0,1,0,1,1,1,0,0,1,1,1,1,SMARCA4
2,SMARCA4 A400A,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,SMARCA4
3,SMARCA4 A400A,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,SMARCA4


In [37]:
# filtered_df = ccle_mutations_filtered[
#     ccle_mutations_filtered.depmap_id == "ACH-001716"
# ]
# point_mutations = list(filtered_df.mutation.values)
# test = construct_anno_features_xon17("ACH-001716", point_mutations, False)

In [38]:
# point_mutations

In [39]:
# test

In [72]:
agg_anno_features_combined_df = anno_features_combined#pd.concat(anno_features_combined).reset_index()
# agg_anno_features_combined_df.rename(columns={"patient_id": "depmap_id"}, inplace=True)
agg_anno_features_combined_df.set_index("input", inplace=True)
agg_anno_features_combined_df

,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
input,,,,,,,,,,,,,,,,,,
NRAS Q61L,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,NRAS
RPTOR D654N,1,1,1,1,0,0,1,0,1,0,1,0,0,1,1,1,1,RPTOR
CTNNB1 G245A,1,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1,CTNNB1
JAK1 I375V,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,JAK1
PIK3C2B R751W,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,PIK3C2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BRCA1 H888Y,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,BRCA1
SMARCA4 A400A,1,1,1,1,1,0,1,0,1,1,1,0,0,1,1,1,1,SMARCA4
SMARCA4 A400A,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,SMARCA4


In [76]:
ccle_mutations_filtered

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation
89,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs
95,TP53,ACH-000001,NaN,other non-conserving,TP53
108,CD79B,ACH-000001,p.L21L,silent,CD79B L21L
109,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L
123,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C
...,...,...,...,...,...
1226149,NTRK1,ACH-002512,p.D270D,silent,NTRK1 D270D
1226178,MLL2,ACH-002512,p.V5399V,silent,MLL2 V5399V
1226212,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y
1226223,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A


In [90]:
# one more round of annotation to confirm
append_df = pd.DataFrame()
for mut in set(ccle_mutations_filtered.mutation) - set(agg_anno_features_combined_df.index):
    res_df = construct_anno_features_xon17_gpd("dummy_patient", [mut], False)
    append_df = pd.concat([append_df, res_df], ignore_index=True)

ERROR:root:Encountered error while processing mutation TGFBR2 S156fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3CA P97fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 G1778G - No columns to parse from file
ERROR:root:Encountered error while processing mutation STAG2 N457fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL W3649L - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL P1349P - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL K1751* - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B G470G - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q3158* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MEF2B P180fs - No columns to parse from file
ERROR:

Point mutation RAD54L is invalid


ERROR:root:Encountered error while processing mutation MLL Q630Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation EP300 K1546fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation DNMT3A G476fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 N134S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL G3902V - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A L141F - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 G10E - No columns to parse from file
ERROR:root:Encountered error while processing mutation PDCD1 P130fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P1097R - No columns to parse from file
ERROR:root:Encountered error while processing mutation ACVR1B R205H - No columns to parse from file
ERROR

Point mutation RAD52 is invalid


ERROR:root:Encountered error while processing mutation EGFR V606fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MYCL1 E256K - No columns to parse from file
ERROR:root:Encountered error while processing mutation RAD51D 292_293MA>MRTA - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A R191R - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 VETV604fs - No columns to parse from file


...

Point mutation CSF3R is invalid


ERROR:root:Encountered error while processing mutation WHSC1 G232S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A R505R - No columns to parse from file
ERROR:root:Encountered error while processing mutation ERBB3 N697fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A LHG14fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 T940fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL S610F - No columns to parse from file
ERROR:root:Encountered error while processing mutation NRAS *190* - No columns to parse from file
ERROR:root:Encountered error while processing mutation PARK2 G328V - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTCH1 P1202fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 K4981K - No columns to parse from file
ERROR:r

Point mutation NTRK2 is invalid


ERROR:root:Encountered error while processing mutation FAM123B H614Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 A815A - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P1137P - No columns to parse from file
ERROR:root:Encountered error while processing mutation FGFR2 T404M - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 T2191K - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM46C E178A - No columns to parse from file


KeyboardInterrupt: 

In [91]:
append_df # Run interrupted to check if there is any update happening

,input,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,KIT I538V,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,KIT
1,KIT I538V,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,KIT
2,DDR1 R123W,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,DDR1
3,MST1R A234S,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,MST1R
4,MPL R123Q,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,1,0,MPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,EZR R526H,0,0,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,EZR
2684,FANCA R951W,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,0,0,FANCA
2685,MTOR T647I,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,MTOR
2686,FANCA Q1128L,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,FANCA


In [99]:
len(set(ccle_mutations_filtered.mutation) - (set(agg_anno_features_combined_df.index)) | set(append_df.input))

22054

In [110]:
# whatever is left ~ 22k mutations

remaining_df = pd.DataFrame()
for mut in set(ccle_mutations_filtered.mutation) - (set(agg_anno_features_combined_df.index) | set(append_df.input)):
    res_df = construct_anno_features_xon17_gpd("dummy_patient", [mut], False)
    remaining_df = pd.concat([remaining_df, res_df], ignore_index=True)

ERROR:root:Encountered error while processing mutation TGFBR2 S156fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3CA P97fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TET2 K1947fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B R857* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 G1778G - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDK4 R246fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation STAG2 N457fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL W3649L - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P1669fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MSH6 K1233fs - No columns to parse from file
ER

ERROR:root:Encountered error while processing mutation PARK2 K76fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL R1686C - No columns to parse from file
ERROR:root:Encountered error while processing mutation RAD52 - No columns to parse from file
ERROR:root:Encountered error while processing mutation RB1 LEHII676fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation GRM3 A269fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation EGFR V606fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MYCL1 E256K - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL T1516S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL F688I - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 A458A - No columns to parse from file
ERROR:root:Enco

ERROR:root:Encountered error while processing mutation BRAF V600K - No columns to parse from file
ERROR:root:Encountered error while processing mutation RAD21 T316fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 W1419* - No columns to parse from file
ERROR:root:Encountered error while processing mutation EWSR1 G490C - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL N3242N - No columns to parse from file
ERROR:root:Encountered error while processing mutation NRAS T20fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation GNAQ - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL L1460L - No columns to parse from file
ERROR:root:Encountered error while processing mutation FUBP1 I298fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BARD1 K208fs - No columns to parse from file
ERROR:root:Enco

ERROR:root:Encountered error while processing mutation MLL2 R1617W - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5A - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P3289T - No columns to parse from file
ERROR:root:Encountered error while processing mutation FANCG G216F - No columns to parse from file
ERROR:root:Encountered error while processing mutation MSH3 E373L - No columns to parse from file
ERROR:root:Encountered error while processing mutation APC V800fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation SPEN R154fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation AXL 378_379D*>E - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 VFPSTA307del - No columns to parse from file
ERROR:root:Encountered error while processing mutation APC G1416fs - No columns to parse from file
ERROR:ro

ERROR:root:Encountered error while processing mutation TP53 S241fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL P2526H - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL T672A - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 R326P - No columns to parse from file
ERROR:root:Encountered error while processing mutation MSH6 R248Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MUTYH S343L - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRCA2 K330fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PARK2 R191W - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM6A MSVS885fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 C1367C - No columns to parse from file
ERRO

...

ERROR:root:Encountered error while processing mutation MLL2 T534M - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL G918G - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 R1244C - No columns to parse from file
ERROR:root:Encountered error while processing mutation PPARG - No columns to parse from file
ERROR:root:Encountered error while processing mutation CEBPA 196_197insHP - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q3278Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q5379R - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R2645* - No columns to parse from file
ERROR:root:Encountered error while processing mutation C17ORF39 R44P - No columns to parse from file
ERROR:root:Encountered error while processing mutation POLD1 G155fs - No columns to parse from file
ERROR:ro

ERROR:root:Encountered error while processing mutation STK11 M51fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 SEESLL2751fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 L1457Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 K1347N - No columns to parse from file
ERROR:root:Encountered error while processing mutation SMARCA4 - No columns to parse from file
ERROR:root:Encountered error while processing mutation ALOX12B L451fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation SMARCA4 YP120fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S2373F - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q152* - No columns to parse from file
ERROR:root:Encountered error while processing mutation NOTCH1 CL1374fs - No columns to parse from fil

ERROR:root:Encountered error while processing mutation CTNNA1 N284fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 KVSI1752del - No columns to parse from file
ERROR:root:Encountered error while processing mutation PARK2 V164A - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCORL1 Q431fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 L323fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ARID1A T1432fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MSH6 V282fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN LK182fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CARD11 E242fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A3615T - No columns to parse from f

ERROR:root:Encountered error while processing mutation WHSC1 R944Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MAP3K1 1315_1316EK>D* - No columns to parse from file
ERROR:root:Encountered error while processing mutation ATM Q1579fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CREBBP 2211_2213QQQ>Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRCA2 D2238fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL D3397D - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL V2338A - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3R1 H450fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ATM R1771fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation EMSY I1006del - No columns to pa

ERROR:root:Encountered error while processing mutation EPHA3 L902fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WT1 R189fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B R180Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation QKI K134fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P1827P - No columns to parse from file
ERROR:root:Encountered error while processing mutation EPHA3 SK644fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation RPTOR F278fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRIP1 N1087fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM46C D277N - No columns to parse from file
ERROR:root:Encountered error while processing mutation FGFR1 L480fs - No columns to parse from file
ER

In [111]:
remaining_df

,input,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,EP300 Q1898R,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,EP300
1,BCL2L2 G50V,1,1,1,1,1,0,1,0,1,1,0,0,0,1,1,1,1,BCL2L2
2,ATM R2436S,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,ATM
3,BCR G420D,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,BCR
4,ESR1 S341L,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,ESR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187,RARA R126Q,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,RARA
7188,RARA H15N,1,1,0,1,0,1,0,1,1,0,0,1,1,1,0,1,1,RARA
7189,RARA H15N,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,RARA
7190,CDKN2C V130A,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,CDKN2C


In [ ]:
## HERE!!!

In [122]:
# mutations without annotations
len(set(ccle_mutations_filtered.mutation) - (set(agg_anno_features_combined_df.index) | set(append_df.input) | set(remaining_df.input)))

12804

In [123]:
len(set(pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv")["input"].unique())), \
len(set(agg_anno_features_combined_df.index) | set(append_df.input) | set(remaining_df.input))

(8483, 16098)

In [125]:
set(pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv")["input"].unique()) - \
(set(agg_anno_features_combined_df.index) | set(append_df.input) | set(remaining_df.input))

set()

In [129]:
# Overwriting existing anno_features_per_mutation_ccle.csv file
concatenated_df = pd.concat([agg_anno_features_combined_df.reset_index(), append_df, remaining_df],ignore_index=True)
concatenated_df

,input,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,NRAS Q61L,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,NRAS
1,RPTOR D654N,1,1,1,1,0,0,1,0,1,0,1,0,0,1,1,1,1,RPTOR
2,CTNNB1 G245A,1,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1,CTNNB1
3,JAK1 I375V,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,JAK1
4,PIK3C2B R751W,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,PIK3C2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,RARA R126Q,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,RARA
18589,RARA H15N,1,1,0,1,0,1,0,1,1,0,0,1,1,1,0,1,1,RARA
18590,RARA H15N,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,RARA
18591,CDKN2C V130A,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,CDKN2C


In [131]:
len(set(concatenated_df["input"]))

16098

In [133]:
len(set(ccle_mutations_filtered["mutation"])) # There are ~ 12k mutations that are missing annotations

28902

In [134]:
concatenated_df.set_index("input", inplace=True, drop=True)

In [135]:
# concatenated_df.to_csv("../data/processed/anno_features_per_mutation_ccle.csv")

In [16]:
concatenated_df = pd.read_csv("../data/processed/anno_features_per_mutation_ccle.csv", index_col = 0)
concatenated_df

,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,fathmm_pred,provean_pred,metasvm_pred,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
input,,,,,,,,,,,,,,,,,,
NRAS Q61L,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,NRAS
RPTOR D654N,1,1,1,1,0,0,1,0,1,0,1,0,0,1,1,1,1,RPTOR
CTNNB1 G245A,1,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1,CTNNB1
JAK1 I375V,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,JAK1
PIK3C2B R751W,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,PIK3C2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RARA R126Q,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,RARA
RARA H15N,1,1,0,1,0,1,0,1,1,0,0,1,1,1,0,1,1,RARA
RARA H15N,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,RARA


In [17]:
ccle_mutations_filtered

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation
89,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs
95,TP53,ACH-000001,NaN,other non-conserving,TP53
108,CD79B,ACH-000001,p.L21L,silent,CD79B L21L
109,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L
123,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C
...,...,...,...,...,...
1226149,NTRK1,ACH-002512,p.D270D,silent,NTRK1 D270D
1226178,MLL2,ACH-002512,p.V5399V,silent,MLL2 V5399V
1226212,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y
1226223,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A


In [18]:
len(ccle_mutations_filtered["DepMap_ID"].unique()), ccle_mutations_filtered.shape

(1744, (36381, 5))

In [19]:
merged_1 = pd.merge(ccle_mutations_filtered, concatenated_df.reset_index(), left_on = "mutation", right_on="input", how="left")
merged_1

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,input,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,...,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TP53,ACH-000001,NaN,other non-conserving,TP53,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CD79B,ACH-000001,p.L21L,silent,CD79B L21L,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C,NOTCH3 R1981C,1.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,NOTCH3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y,BRCA1 H888Y,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BRCA1
62308,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,SMARCA4 A400A,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,SMARCA4
62309,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,SMARCA4 A400A,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4
62310,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,SMARCA4 A400A,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4


In [20]:
merged_1.drop("input", axis = 1, inplace=True)
merged_1.isna().sum()

Hugo_Symbol                   0
DepMap_ID                     0
Protein_Change              987
Variant_annotation            0
mutation                      0
sift_pred                 16867
sift4g_pred               16867
lrt_pred                  16867
mutationtaster_pred       16867
mutationassessor_pred     16867
fathmm_pred               16867
provean_pred              16867
metasvm_pred              16867
m_cap_pred                16867
primateai_pred            16867
deogen2_pred              16867
bayesdel_addaf_pred       16867
bayesdel_noaf_pred        16867
clinpred_pred             16867
list_s2_pred              16867
fathmm_mkl_coding_pred    16867
fathmm_xf_coding_pred     16867
gene                      16867
dtype: int64

In [21]:
# For those point mutations without annotations
merged_1.fillna(0, inplace=True)

In [22]:
merged_1

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,...,m_cap_pred,primateai_pred,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene
0,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,TP53,ACH-000001,0,other non-conserving,TP53,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,CD79B,ACH-000001,p.L21L,silent,CD79B L21L,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,NOTCH3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BRCA1
62308,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,SMARCA4
62309,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4
62310,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4


In [23]:
merged_1["HGVSp"] = merged_1["mutation"].apply(lambda x: "p." + x.split(" ")[1] if len(x.split(" ")) >= 2 else "p.")

In [24]:
len(merged_1["HGVSp"].unique())

25538

In [25]:
len(merged_1["mutation"].unique()) # this includes gene name as well

28902

### ClinVar annotations

In [26]:
import tempfile
import logging
import subprocess
ANNOTATION_SCRIPT_PATH_CLINVAR = "../script/goAAtoGv2_clinvar.sh"

In [32]:
results = []
for mutation in list(merged_1["mutation"]):
    try:
        # Run annotation script within a temp file and extract features as DataFrame
        with tempfile.TemporaryDirectory() as tmpdirname:
            input_file_path = tmpdirname + "anno_input.txt"
            with open(input_file_path, "w+") as input_file:
                mutation_cleaned = [part for part in mutation.split(" ") if part]
                input_file.write(":p.".join(mutation_cleaned))
                input_file.write("\n")

            # Execute script
            cmd = "bash {0} {1}".format(ANNOTATION_SCRIPT_PATH_CLINVAR, input_file_path)

            logging.info(f"Executing command {cmd}")
            subprocess.call(cmd, shell=True, executable="/bin/bash")
            out_file_path = f"{input_file_path}.annot.hg38_finalannot.txt"
            res = pd.read_table(out_file_path)
    # Some inputs lead to errors, such as "PTEN loss" - ignore and continue processing
    except Exception as e:
        logging.error(
            f"Encountered error while processing mutation {mutation} - {e}"
        )
        continue
    res["input"] = mutation
    res = res[~res.duplicated()]
    results.append(res)
clinvar_annot_df = pd.concat(results)
clinvar_annot_df.set_index(["input"], inplace=True)
clinvar_annot_df.drop(columns=["Otherinfo1"], inplace=True)

ERROR:root:Encountered error while processing mutation TSC2 F297fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3R1 - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 K1074E - No columns to parse from file
ERROR:root:Encountered error while processing mutation CEBPA 186_189PPPP>P - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A R29* - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5C E1101del - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B G820G - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 D560H - No columns to parse from file
ERROR:root:Encountered error while processing mutation ASXL1 L1393fs - No columns to parse from file
ERRO

ERROR:root:Encountered error while processing mutation EGFR ELREA701del - No columns to parse from file
ERROR:root:Encountered error while processing mutation MED12 Q1917del - No columns to parse from file
ERROR:root:Encountered error while processing mutation CBL 36_36H>HH - No columns to parse from file
ERROR:root:Encountered error while processing mutation DDR2 -563fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation DDR2 LT563fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL S100S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL T170A - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL A1032A - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL P1798P - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL I1926I - No columns to parse from file
ERROR:

ERROR:root:Encountered error while processing mutation TP53 - No columns to parse from file
ERROR:root:Encountered error while processing mutation CTNNB1 - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 FC134fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation KEAP1 119_120GM>V - No columns to parse from file
ERROR:root:Encountered error while processing mutation JAK2 TK455fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 - No columns to parse from file
ERROR:root:Encountered error while processing mutation POLD1 G155fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation APC AK1591fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation EZH2 R690H - No columns to parse from file
ERROR:root:Encountered error while processing mutation BRD4 YLQQ931del - No columns to parse from file
ERROR:root:Encoun

ERROR:root:Encountered error while processing mutation NOTCH1 V2473fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation RNF43 R330fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A 18_19insAT - No columns to parse from file
ERROR:root:Encountered error while processing mutation ARID1A P1922fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 P152fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDH1 PIFVPP484del - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 - No columns to parse from file
ERROR:root:Encountered error while processing mutation CEBPA 186_189PPPP>P - No columns to parse from file
ERROR:root:Encountered error while processing mutation SF3B1 *968fs - No columns to parse from f

ERROR:root:Encountered error while processing mutation MLL2 S717Y - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B G1031D - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN P204fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ALK C928fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 R420fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation REL I543fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 S1093L - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 P191del - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAS F133fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ETV6 EKMSR392fs - No columns to parse from file


...

ERROR:root:Encountered error while processing mutation FAM46C N159N - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A R400S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL P1868fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL S2108R - No columns to parse from file
ERROR:root:Encountered error while processing mutation ETV6 S203fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation KRAS K185fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 E5158* - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S4907S - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A2139T - No columns to parse from file
ERROR:root:Encountered error while processing mutation ERBB3 V411fs - No columns to parse from file
ERROR:ro

ERROR:root:Encountered error while processing mutation PTEN - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN E288fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL R3935C - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5A K1199fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3C2G - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 D5040D - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P4977L - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 Q3943Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P3494P - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P2198fs - No columns to parse from file
ERROR:root:Encounte

ERROR:root:Encountered error while processing mutation APC E1554fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CTNNA1 - No columns to parse from file
ERROR:root:Encountered error while processing mutation CSF1R - No columns to parse from file
ERROR:root:Encountered error while processing mutation PRDM1 K209fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TEK - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTCH1 P1307fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCOR E1382fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL R721Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5A - No columns to parse from file
ERROR:root:Encountered error while proces

ERROR:root:Encountered error while processing mutation BRAF - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN2A G85fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCORL1 S1679fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A L76L - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5A K1199fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R5220R - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A4594fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A3417V - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P648fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P634S - No columns to parse from file
ERROR:root:

KeyboardInterrupt: 

In [34]:
len(results)

32433

In [35]:
clinvar_annot_df = pd.concat(results)
clinvar_annot_df.set_index(["input"], inplace=True)
clinvar_annot_df.drop(columns=["Otherinfo1"], inplace=True)

In [36]:
clinvar_annot_df

,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,LRT_score,LRT_converted_rankscore,LRT_pred,MutationTaster_score,...,GERP++_RS_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons30way_mammalian,phastCons30way_mammalian_rankscore,CLNALLELEID,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG
input,,,,,,,,,,,,,,,,,,,,,
CD79B L21L,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
SOX9 L3L,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
NOTCH3 R1981C,0.0,0.913,D,0.0,0.928,D,0.01,0.302,N,1,...,0.336,1.0,0.716,0.393,0.256,.,.,.,.,.
PPP2R1A E100K,0.001,0.785,D,0.012,0.639,D,0.0,0.629,D,1.0,...,0.536,1.0,0.716,0.995,0.577,.,.,.,.,.
PPP2R1A S120S,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KDM5C R1456W,0.01,0.575,D,0.014,0.631,D,0.145,0.182,U,1,...,0.343,1.0,0.716,0.999,0.704,.,.,.,.,.
KDM5C R1456W,0.001,0.785,D,0.002,0.794,D,0.672,0.103,U,1,...,0.452,0.963,0.338,1.0,0.863,.,.,.,.,.
KDM5C R1456W,0.01,0.575,D,0.014,0.631,D,0.145,0.182,U,1,...,0.343,1.0,0.716,0.999,0.704,.,.,.,.,.


In [37]:
clinvar_annot_df.CLNSIG.value_counts()

.                                                     16301
Pathogenic                                             6660
Pathogenic|drug_response|other                         5120
Uncertain_significance                                 2074
Pathogenic/Likely_pathogenic                           1625
Conflicting_interpretations_of_pathogenicity           1440
Likely_benign                                           805
Likely_pathogenic                                       465
Benign/Likely_benign                                    203
Benign                                                  131
not_provided                                             46
Conflicting_interpretations_of_pathogenicity|other       24
drug_response                                             5
Pathogenic/Likely_pathogenic|other                        5
Uncertain_significance|drug_response                      3
Likely_pathogenic|other                                   2
Pathogenic|risk_factor                  

In [41]:
remaining_mut = list(set(merged_1["mutation"]) - set(clinvar_annot_df.index))
len(set(merged_1["mutation"]) - set(clinvar_annot_df.index))

13558

In [42]:
# one more round with all the mutations that were missed out before the earlier interrupt
results2 = []
for mutation in remaining_mut:
    try:
        # Run annotation script within a temp file and extract features as DataFrame
        with tempfile.TemporaryDirectory() as tmpdirname:
            input_file_path = tmpdirname + "anno_input.txt"
            with open(input_file_path, "w+") as input_file:
                mutation_cleaned = [part for part in mutation.split(" ") if part]
                input_file.write(":p.".join(mutation_cleaned))
                input_file.write("\n")

            # Execute script
            cmd = "bash {0} {1}".format(ANNOTATION_SCRIPT_PATH_CLINVAR, input_file_path)

            logging.info(f"Executing command {cmd}")
            subprocess.call(cmd, shell=True, executable="/bin/bash")
            out_file_path = f"{input_file_path}.annot.hg38_finalannot.txt"
            res = pd.read_table(out_file_path)
    # Some inputs lead to errors, such as "PTEN loss" - ignore and continue processing
    except Exception as e:
        logging.error(
            f"Encountered error while processing mutation {mutation} - {e}"
        )
        continue
    res["input"] = mutation
    res = res[~res.duplicated()]
    results2.append(res)

ERROR:root:Encountered error while processing mutation PBRM1 V1179fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 H796Y - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 K1349M - No columns to parse from file
ERROR:root:Encountered error while processing mutation SYK QG31fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 A1277G - No columns to parse from file
ERROR:root:Encountered error while processing mutation TBX3 R427Q - No columns to parse from file
ERROR:root:Encountered error while processing mutation SGK1 - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL R3291T - No columns to parse from file
ERROR:root:Encountered error while processing mutation SETD2 N1396fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ZNF217 L367fs - No columns to parse from file
ERROR:root:

ERROR:root:Encountered error while processing mutation MLL2 H1071H - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL S3911S - No columns to parse from file
ERROR:root:Encountered error while processing mutation KDM5C W134del - No columns to parse from file
ERROR:root:Encountered error while processing mutation TNFRSF14 - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R5225C - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A L141F - No columns to parse from file
ERROR:root:Encountered error while processing mutation NPM1 AK249fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3C2G - No columns to parse from file
ERROR:root:Encountered error while processing mutation POLD1 GV64fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3R1 QF92del - No columns to parse from file
ERROR:root

ERROR:root:Encountered error while processing mutation MYC - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCL2 - No columns to parse from file
ERROR:root:Encountered error while processing mutation SMARCA4 YP120fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NOTCH3 D2010fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 A262A - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 L1245L - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 D1309N - No columns to parse from file
ERROR:root:Encountered error while processing mutation PIK3R1 T91del - No columns to parse from file
ERROR:root:Encountered error while processing mutation PALB2 M296fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation RB1 A14fs - No columns to parse from file
ERROR:root:Enc

ERROR:root:Encountered error while processing mutation KIT KS193fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 E204fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 L978L - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 H1365Y - No columns to parse from file
ERROR:root:Encountered error while processing mutation MET - No columns to parse from file
ERROR:root:Encountered error while processing mutation MRE11A R80I - No columns to parse from file
ERROR:root:Encountered error while processing mutation ALOX12B L340fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NOTCH1 P2340fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MPL L571fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P530H - No columns to parse from file
ERROR:root:E

ERROR:root:Encountered error while processing mutation ARID1A S409fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R5389R - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 G4397V - No columns to parse from file
ERROR:root:Encountered error while processing mutation RB1 AAT10del - No columns to parse from file
ERROR:root:Encountered error while processing mutation ARID1A V1024fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PPP2R2A C13fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 H179fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CARD11 R555fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation DDR1 G29del - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1 V435I - No columns to parse from file


...

ERROR:root:Encountered error while processing mutation MLL2 K4843N - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL T993A - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL S912F - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P772H - No columns to parse from file
ERROR:root:Encountered error while processing mutation CTCF -146fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation IDH1 F32fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLH1 S470fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM123B Q675H - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN Q17del - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 G117fs - No columns to parse from file
ERROR:root:Enc

ERROR:root:Encountered error while processing mutation MLL2 N2030H - No columns to parse from file
ERROR:root:Encountered error while processing mutation MCL1 A19fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R2068R - No columns to parse from file
ERROR:root:Encountered error while processing mutation ATRX 228_229NL>K - No columns to parse from file
ERROR:root:Encountered error while processing mutation FLT1 K1153fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation ARID1A A1517fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation CBFB L88fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MYCL1 G16G - No columns to parse from file
ERROR:root:Encountered error while processing mutation FAM46C I94fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 V1061fs - No columns to parse from file
ERR

ERROR:root:Encountered error while processing mutation DNMT3A L636del - No columns to parse from file
ERROR:root:Encountered error while processing mutation SOX9 S414fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P674P - No columns to parse from file
ERROR:root:Encountered error while processing mutation PBRM1 I279fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P565L - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF1 D2635fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 R4904* - No columns to parse from file
ERROR:root:Encountered error while processing mutation POLE N751fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation C17ORF39 R44P - No columns to parse from file
ERROR:root:Encountered error while processing mutation CDKN1B KGAC96fs - No columns to parse from file
ER

ERROR:root:Encountered error while processing mutation FAM123B L1052L - No columns to parse from file
ERROR:root:Encountered error while processing mutation RB1 L512fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 S1384* - No columns to parse from file
ERROR:root:Encountered error while processing mutation PTEN M199del - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 P759P - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 DD48fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCL6 R459fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation APC RK1790fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation BCOR G400C - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL V2525V - No columns to parse from file
ERROR:

ERROR:root:Encountered error while processing mutation CSF1R G80fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NF2 K228fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation FLCN N236fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation NBN V271fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 D1825N - No columns to parse from file
ERROR:root:Encountered error while processing mutation TET2 L1332H - No columns to parse from file
ERROR:root:Encountered error while processing mutation WHSC1L1 C949C - No columns to parse from file
ERROR:root:Encountered error while processing mutation TP53 HLIRVE193fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation PRDM1 D246fs - No columns to parse from file
ERROR:root:Encountered error while processing mutation MLL2 S1229F - No columns to parse from file
ERRO

In [43]:
clinvar_annot_df2 = pd.concat(results2)
clinvar_annot_df2.set_index(["input"], inplace=True)
clinvar_annot_df2.drop(columns=["Otherinfo1"], inplace=True)
clinvar_annot_df2

,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,LRT_score,LRT_converted_rankscore,LRT_pred,MutationTaster_score,...,GERP++_RS_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons30way_mammalian,phastCons30way_mammalian_rankscore,CLNALLELEID,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG
input,,,,,,,,,,,,,,,,,,,,,
SLC34A2 V607I,1.0,0.01,T,1.0,0.012,T,0.082,0.208,N,1,...,0.007,0.0,0.064,0.002,0.07,.,.,.,.,.
PIK3C2B A1460T,0.061,0.371,T,0.157,0.345,T,0.017,0.278,N,1,...,0.266,0.389,0.261,0.962,0.435,.,.,.,.,.
CARD11 S694S,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
MAP2K4 R41C,0.0,0.913,D,0.066,0.445,T,0.0,0.843,D,1,...,0.807,1.0,0.716,1.0,0.863,.,.,.,.,.
PIK3CA H1048R,1.0,0.01,T,0.638,0.07,T,0.0,0.843,D,1,...,0.99,1.0,0.716,0.996,0.595,443385,not_provided,MedGen:CN517202,"criteria_provided,_single_submitter",Likely_pathogenic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NFKBIA L229L,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
ALK Y1604H,0.004,0.654,D,0.116,0.365,T,0.026,0.259,N,0.999,...,0.378,0.602,0.278,0.015,0.131,.,.,.,.,.
ERBB3 V104L,0.131,0.371,T,0.274,0.483,T,0.001,0.432,N,1.0,...,0.644,1.0,0.716,0.71,0.312,.,.,.,.,.


In [44]:
clinvar_annot_df_overall = pd.concat([clinvar_annot_df, clinvar_annot_df2], axis = 0)
clinvar_annot_df_overall

,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,LRT_score,LRT_converted_rankscore,LRT_pred,MutationTaster_score,...,GERP++_RS_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons30way_mammalian,phastCons30way_mammalian_rankscore,CLNALLELEID,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG
input,,,,,,,,,,,,,,,,,,,,,
CD79B L21L,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
SOX9 L3L,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
NOTCH3 R1981C,0.0,0.913,D,0.0,0.928,D,0.01,0.302,N,1,...,0.336,1.0,0.716,0.393,0.256,.,.,.,.,.
PPP2R1A E100K,0.001,0.785,D,0.012,0.639,D,0.0,0.629,D,1.0,...,0.536,1.0,0.716,0.995,0.577,.,.,.,.,.
PPP2R1A S120S,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NFKBIA L229L,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
ALK Y1604H,0.004,0.654,D,0.116,0.365,T,0.026,0.259,N,0.999,...,0.378,0.602,0.278,0.015,0.131,.,.,.,.,.
ERBB3 V104L,0.131,0.371,T,0.274,0.483,T,0.001,0.432,N,1.0,...,0.644,1.0,0.716,0.71,0.312,.,.,.,.,.


In [45]:
clinvar_annot_df_overall.CLNSIG.value_counts()

.                                                     24598
Pathogenic                                             6807
Pathogenic|drug_response|other                         5120
Uncertain_significance                                 2959
Pathogenic/Likely_pathogenic                           1657
Conflicting_interpretations_of_pathogenicity           1634
Likely_benign                                          1214
Likely_pathogenic                                       527
Benign/Likely_benign                                    248
Benign                                                  152
not_provided                                             56
Conflicting_interpretations_of_pathogenicity|other       24
Pathogenic/Likely_pathogenic|other                        6
drug_response                                             5
Uncertain_significance|drug_response                      3
Likely_pathogenic|other                                   2
Pathogenic|risk_factor                  

In [46]:
clinvar_annot_df_overall.to_csv("../data/processed/clinvar_anno_features_per_mutation_ccle.csv")

### Output: Revised Annovar Xon17 score

In [159]:
merged_1["1plusxon17_score"] = 1 + merged_1[CATEGORICAL_COLUMNS].sum(axis=1)/17
merged_1

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,...,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene,HGVSp,1plusxon17_score
0,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs,0,0,0,0,0,...,0,0,0,0,0,0,0,0,p.F297fs,1.000000
1,TP53,ACH-000001,0,other non-conserving,TP53,0,0,0,0,0,...,0,0,0,0,0,0,0,0,p.,1.000000
2,CD79B,ACH-000001,p.L21L,silent,CD79B L21L,0,0,0,0,0,...,0,0,0,0,0,0,0,0,p.L21L,1.000000
3,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L,0,0,0,0,0,...,0,0,0,0,0,0,0,0,p.L3L,1.000000
4,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C,1,1,0,1,0,...,0,0,1,1,1,1,1,NOTCH3,p.R1981C,1.647059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y,0,0,0,0,1,...,0,0,0,0,0,0,0,BRCA1,p.H888Y,1.176471
62308,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1,1,1,1,1,...,1,0,0,1,1,1,1,SMARCA4,p.A400A,1.764706
62309,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1,1,1,1,1,...,1,1,1,1,1,1,1,SMARCA4,p.A400A,2.000000
62310,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1,1,1,1,1,...,1,1,1,1,1,1,1,SMARCA4,p.A400A,2.000000


In [161]:
patient_gene_matrix_xon17 = merged_1.pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17.fillna(0, inplace=True)
patient_gene_matrix_xon17

Hugo_Symbol,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,APC,AR,ARAF,...,U2AF1,VEGFA,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.294118,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000
ACH-000004,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.235294
ACH-000005,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002508,0.0,1.0,0.0,0.0,0.0,1.352941,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.529412,0.0,0.0,0.000000
ACH-002509,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
ACH-002510,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [162]:
for g in GENES_324:
    if g not in patient_gene_matrix_xon17.columns:
        patient_gene_matrix_xon17[g] = 0
patient_gene_matrix_xon17.shape

(1744, 324)

In [167]:
# patient_gene_matrix_xon17[GENES_324].reset_index().rename(columns={"DepMap_ID": "depmap_id"}).to_csv("../data/processed/ccle_anno_features_xon17.csv", index=False)

### Variant Annotations from GPD

In [148]:
# Load intermediate GPD files for NPC and PC

In [47]:
npc_mutations = pd.read_csv("../data/processed/ccle_21q3_gpd_results/ccle_21q3_mutation_npc.tsv", sep="\t")
npc_mutations

,Hugo_Symbol,Gene,Chromosome,Start_Position,End_Position,Variant_Classification,Variant_Type,HGVSc,HGVSp,agg_sample_id,mut_freq
0,SPRY4,ENSG00000187678,5,141694722,141694722,Splice_Site,SNP,NaN,NaN,ACH-000036_ACH-000095_ACH-000152_ACH-000157_AC...,67
1,LMBRD1,ENSG00000168216,6,70411439,70411440,Splice_Site,INS,NaN,NaN,ACH-000076_ACH-000108_ACH-000132_ACH-000144_AC...,58
2,TP53,ENSG00000141510,17,7577538,7577538,Missense_Mutation,SNP,NaN,NaN,ACH-000001_ACH-000013_ACH-000030_ACH-000042_AC...,54
3,TRAPPC9,ENSG00000167632,8,140898214,140898215,Splice_Site,INS,NaN,NaN,ACH-000003_ACH-000009_ACH-000015_ACH-000034_AC...,48
4,CTD-2587H24.4,NaN,19,55671383,55671383,Splice_Site,SNP,NaN,NaN,ACH-000128_ACH-000172_ACH-000206_ACH-000246_AC...,47
...,...,...,...,...,...,...,...,...,...,...,...
21558,ZZEF1,ENSG00000074755,17,3935418,3935418,Splice_Site,SNP,NaN,NaN,ACH-001517,1
21559,ZZEF1,ENSG00000074755,17,3947517,3947517,Splice_Site,SNP,NaN,NaN,ACH-000805,1
21560,ZZEF1,ENSG00000074755,17,3989950,3989950,Splice_Site,SNP,NaN,NaN,ACH-002100,1
21561,ZZEF1,ENSG00000074755,17,3994125,3994125,Splice_Site,SNP,NaN,NaN,ACH-001524,1


In [48]:
npc_mutations["HGVSp"].value_counts(dropna=False)

NaN    21563
Name: HGVSp, dtype: int64

In [49]:
npc_mutations["Variant_Classification"].value_counts(dropna=False)

Splice_Site                 17576
De_novo_Start_OutOfFrame     3199
Intron                        289
Stop_Codon_Del                130
Start_Codon_Del                85
IGR                            77
5'Flank                        67
3'UTR                          39
Stop_Codon_Ins                 37
5'UTR                          30
Start_Codon_Ins                26
Missense_Mutation               5
Frame_Shift_Del                 2
Nonsense_Mutation               1
Name: Variant_Classification, dtype: int64

In [50]:
pc_mutations = pd.read_csv("../data/processed/ccle_21q3_gpd_results/ccle_21q3_mutation_pc_pos.tsv", sep="\t")
pc_mutations

,Chromosome,Start_Position,End_Position,Variant_Classification,Variant_Type,Hugo_Symbol,Gene,HGVSc,HGVSp,prot_start_pos,prot_end_pos,agg_sample_id,mut_freq
0,7,140453136,140453136,Missense_Mutation,SNP,BRAF,ENSG00000157764,c.1799T>A,p.V600E,600,600,ACH-000008_ACH-000014_ACH-000052_ACH-000112_AC...,114
1,4,952220,952220,Missense_Mutation,SNP,TMEM175,ENSG00000127419,c.1451A>C,p.H484P,484,484,ACH-000024_ACH-000030_ACH-000045_ACH-000056_AC...,88
2,16,89751858,89751895,Frame_Shift_Del,DEL,RP11-368I7.4,NaN,c.150_187delAGACCAGCTCGGTCGGGCCCAAACCACCGCCCAC...,p.DQLGRAQTTAHCG51fs,51,51,ACH-000087_ACH-000095_ACH-000128_ACH-000143_AC...,86
3,20,31035566,31035566,Missense_Mutation,SNP,C20orf112,NaN,c.1144A>C,p.T382P,382,382,ACH-000100_ACH-000152_ACH-000157_ACH-000247_AC...,82
4,11,55861307,55861307,Frame_Shift_Del,DEL,OR8I2,ENSG00000172154,c.524delA,p.H175fs,175,175,ACH-000002_ACH-000006_ACH-000039_ACH-000094_AC...,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...
735121,19,14184004,14184004,Frame_Shift_Del,DEL,hsa-mir-1199,NaN,c.309delG,p.Q103fs,103,103,ACH-000988,1
735122,19,14184104,14184104,Frame_Shift_Del,DEL,hsa-mir-1199,NaN,c.409delC,p.P138fs,138,138,ACH-000998,1
735123,19,14184596,14184596,Missense_Mutation,SNP,hsa-mir-1199,NaN,c.475C>T,p.R159C,159,159,ACH-000442,1
735124,19,14184682,14184682,Missense_Mutation,SNP,hsa-mir-1199,NaN,c.561C>A,p.S187R,187,187,ACH-000483,1


In [51]:
pc_mutations["Variant_Classification"].value_counts(dropna=False)

Missense_Mutation    627709
Nonsense_Mutation     37165
Frame_Shift_Del       28603
Splice_Site           19345
Frame_Shift_Ins       13076
In_Frame_Del           5706
In_Frame_Ins           1423
Start_Codon_SNP        1262
Nonstop_Mutation        837
Name: Variant_Classification, dtype: int64

In [52]:
# To get PIU vs LU, we used the locations in ptm_pfam_combine.csv which is used in the GPD implementation
ptm_pfam_df = pd.read_csv("/data/ajayago/druid/datasets/ptm_pfam_combine.csv", index_col = 0)
ptm_pfam_df

,uniprot_accession,start_position,end_position,center_position,unit_name,gene_name,gene_id,unit_label
1,A0A024RBG1,17,144,80,NUDIX,NUDT4,ENSG00000173598,Domain
2,A0A024RBG1,17,144,80,NUDIX,NUDT4B,ENSG00000173598,Domain
3,A0A024RBG1,17,144,80,NUDIX,NUDT4,ENSG00000177144,Domain
4,A0A024RBG1,17,144,80,NUDIX,NUDT4B,ENSG00000177144,Domain
5,A0A075B6H9,25,119,72,V-set,IGLV4-69,ENSG00000211637,Domain
...,...,...,...,...,...,...,...,...
394261,Q9Y6Z7,196,206,201,acety,COLEC10,ENSG00000184374,PTM
394262,Q9Y6Z7,165,272,218,Lectin_C,COLEC10,ENSG00000184374,Domain
394263,S4R3P1,1,24,12,Humanin,MTRNR2L13,ENSG00000270394,Family
394264,S4R3Y5,1,24,12,Humanin,MTRNR2L11,ENSG00000270188,Family


In [53]:
GPD_unit = []
for idx, row in pc_mutations.iterrows():
    subset_ptm = ptm_pfam_df[ptm_pfam_df.gene_id == row["Gene"]]
    x = "LU"
    for idx, r in subset_ptm.iterrows():
        if (row["prot_start_pos"] >= r["start_position"]) & (row["prot_start_pos"] <= r["end_position"]) \
    | (row["prot_end_pos"] >= r["start_position"]) & (row["prot_end_pos"] <= r["end_position"]):
            x = "PIU"
            break
    GPD_unit.append(x)

pc_mutations["GPD_unit"] = GPD_unit

In [54]:
pc_mutations["GPD_unit"].value_counts()

PIU    411879
LU     323247
Name: GPD_unit, dtype: int64

In [55]:
piu_mutations = set(pc_mutations[pc_mutations["GPD_unit"] == "PIU"]["Hugo_Symbol"] + " " + pc_mutations[pc_mutations["GPD_unit"] == "PIU"]["HGVSp"].apply(lambda x: x.split("p.")[1]))
lu_mutations = set(pc_mutations[pc_mutations["GPD_unit"] == "LU"]["Hugo_Symbol"] + " " + pc_mutations[pc_mutations["GPD_unit"] == "LU"]["HGVSp"].apply(lambda x: x.split("p.")[1]))
len(piu_mutations), len(lu_mutations)

(410753, 322315)

In [56]:
# Map each point mutation to PIU/LU or NCU
GPD_unit_merged1 = []
for idx, row in merged_1.iterrows():
    if row["mutation"] in (piu_mutations):
        GPD_unit_merged1.append("PIU")
    elif row["mutation"] in (lu_mutations):
        GPD_unit_merged1.append("LU")
    else:
        GPD_unit_merged1.append("NCU")

In [57]:
merged_1["GPD_unit"] = GPD_unit_merged1

In [58]:
merged_1

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,...,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene,HGVSp,GPD_unit
0,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,p.F297fs,PIU
1,TP53,ACH-000001,0,other non-conserving,TP53,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,p.,NCU
2,CD79B,ACH-000001,p.L21L,silent,CD79B L21L,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,p.L21L,NCU
3,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,p.L3L,NCU
4,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,NOTCH3,p.R1981C,PIU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BRCA1,p.H888Y,LU
62308,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,SMARCA4,p.A400A,NCU
62309,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4,p.A400A,NCU
62310,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4,p.A400A,NCU


### Output: GPD based Annovar annotation

In [180]:
patient_gene_matrix_xon17_piu = merged_1[merged_1.GPD_unit == "PIU"].pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17_piu.fillna(0, inplace=True)
for g in GENES_324:
    if g not in patient_gene_matrix_xon17_piu.columns:
        patient_gene_matrix_xon17_piu[g] = 0

patient_gene_matrix_xon17_piu

Hugo_Symbol,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,APC,AR,ARAF,...,ZNF217,ZNF703,C17ORF39,EMSY,FAM123B,MLL,MLL2,MYCL1,PTEN,TERC
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.294118,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
ACH-000004,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
ACH-000005,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002467,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
ACH-002508,0.0,0.0,0.0,0.0,0.0,1.352941,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
ACH-002509,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0


In [181]:
patient_gene_matrix_xon17_lu = merged_1[merged_1.GPD_unit == "LU"].pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17_lu.fillna(0, inplace=True)
for g in GENES_324:
    if g not in patient_gene_matrix_xon17_lu.columns:
        patient_gene_matrix_xon17_lu[g] = 0
patient_gene_matrix_xon17_lu

Hugo_Symbol,ABL1,ACVR1B,ALK,ALOX12B,APC,AR,ARAF,ARFRP1,ARID1A,ASXL1,...,HRAS,MAP2K2,MLL,MLL2,MYCL1,PRKAR1A,RAC1,RAF1,TERC,WT1
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
ACH-000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
ACH-002508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
ACH-002509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
patient_gene_matrix_xon17_ncu = merged_1[merged_1.GPD_unit == "NCU"].pivot_table(index="DepMap_ID", columns="Hugo_Symbol", values="1plusxon17_score", aggfunc="max")
patient_gene_matrix_xon17_ncu.fillna(0, inplace=True)
for g in GENES_324:
    if g not in patient_gene_matrix_xon17_ncu.columns:
        patient_gene_matrix_xon17_ncu[g] = 0
patient_gene_matrix_xon17_ncu

Hugo_Symbol,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,APC,AR,ARAF,...,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703,SDHC,TERC
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ACH-000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002508,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ACH-002509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
ACH-002510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [188]:
patient_gene_matrix_xon17_piu.shape, patient_gene_matrix_xon17_lu.shape, patient_gene_matrix_xon17_ncu.shape

((1709, 324), (1570, 324), (1620, 324))

In [190]:
len(set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_piu.index))

35

In [192]:
# Add in missing patient IDs in each matrix
patient_gene_matrix_xon17_piu.reset_index(inplace=True)
patient_gene_matrix_xon17_lu.reset_index(inplace=True)
patient_gene_matrix_xon17_ncu.reset_index(inplace=True)
for t in set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_piu.DepMap_ID):
    patient_gene_matrix_xon17_piu = (patient_gene_matrix_xon17_piu.append({"DepMap_ID": t}, ignore_index=True))
patient_gene_matrix_xon17_piu.set_index("DepMap_ID", inplace=True)    
for t in set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_lu.DepMap_ID):
    patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
patient_gene_matrix_xon17_lu.set_index("DepMap_ID", inplace=True)
for t in set(patient_gene_matrix_xon17.index) - set(patient_gene_matrix_xon17_ncu.DepMap_ID):
    patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
patient_gene_matrix_xon17_ncu.set_index("DepMap_ID", inplace=True)  

<ipython-input-192-5483b5cdd229>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_piu = (patient_gene_matrix_xon17_piu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_piu = (patient_gene_matrix_xon17_piu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_piu = (patient_gene_matrix_xon17_piu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_lu = (patient_gene_matrix_xon17_lu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))
<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

<ipython-input-192-5483b5cdd229>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patient_gene_matrix_xon17_ncu = (patient_gene_matrix_xon17_ncu.append({"DepMap_ID": t}, ignore_index=True))


In [193]:
patient_gene_matrix_xon17_piu.shape, patient_gene_matrix_xon17_lu.shape, patient_gene_matrix_xon17_ncu.shape

((1744, 325), (1744, 325), (1744, 325))

In [195]:
patient_gene_matrix_xon17_piu.drop("index", axis =1, inplace=True)
patient_gene_matrix_xon17_lu.drop("index", axis =1, inplace=True)
patient_gene_matrix_xon17_ncu.drop("index", axis =1, inplace=True)

In [197]:
patient_gene_matrix_xon17_piu.fillna(0, inplace=True)
patient_gene_matrix_xon17_lu.fillna(0, inplace=True)
patient_gene_matrix_xon17_ncu.fillna(0, inplace=True)

In [199]:
patient_gene_matrix_xon17_piu.loc[patient_gene_matrix_xon17.index][GENES_324].to_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_piu_annotated_df.csv")

In [200]:
patient_gene_matrix_xon17_lu.loc[patient_gene_matrix_xon17.index][GENES_324].to_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_lu_annotated_df.csv")

In [201]:
patient_gene_matrix_xon17_ncu.loc[patient_gene_matrix_xon17.index][GENES_324].to_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_ncu_annotated_df.csv")

In [203]:
pd.read_csv("../data/processed/xon17_gpd_annotations/ccle_21q3_piu_annotated_df.csv").set_index("DepMap_ID")

,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,APC,AR,ARAF,...,U2AF1,VEGFA,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.294118,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000004,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000005,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002508,0.0,0.0,0.0,0.0,0.0,1.352941,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-002509,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-002510,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
merged_1.to_csv("../data/processed/ccle_21q3_annovar_gpd_annot_per_patient_per_mutation.csv", index=False)

#### Combine Clinvar annotation, Annovar and GPD annotations

In [64]:
merged_clinvar_df = pd.merge(merged_1, clinvar_annot_df_overall.reset_index(), left_on="mutation", right_on="input", how="outer").groupby("mutation").aggregate(max)

In [73]:
merged_clinvar_df[(merged_clinvar_df.CLNSIG != ".")&(~merged_clinvar_df.CLNSIG.isna())&(merged_clinvar_df.CLNSIG=="Pathogenic")][["GPD_unit", "CLNSIG", ]]

,GPD_unit,CLNSIG
mutation,,
AKT1 E17K,PIU,Pathogenic
ALK R1275Q,PIU,Pathogenic
ALOX12B P422L,PIU,Pathogenic
APC E1309*,PIU,Pathogenic
APC E1554*,PIU,Pathogenic
...,...,...
VHL S65W,PIU,Pathogenic
VHL W88*,PIU,Pathogenic
WT1 Q155*,PIU,Pathogenic


In [65]:
merged_clinvar_df.groupby(["GPD_unit", "CLNSIG"]).aggregate("count")

Hugo_Symbol  \
GPD_unit CLNSIG                                                            
LU       .                                                          4850   
         Benign                                                       16   
         Benign/Likely_benign                                          7   
         Conflicting_interpretations_of_pathogenicity                141   
         Likely_benign                                                28   
         Likely_pathogenic                                            49   
         Pathogenic                                                  130   
         Pathogenic/Likely_pathogenic                                 22   
         Uncertain_significance                                      753   
         not_provided                                                  8   
NCU      .                                                          6559   
         Benign                                                       23   
         Benign/Likely_benign                                         87   
         Conflicting_interpretations_of_pathogenicity                 95   
         Likely_benign                                               961   
         Likely_risk_allele                                            1   
         Uncertain_significance                                       59   
         not_provided                                                  7   
PIU      .                                                          8609   
         Benign                                                       15   
         Benign/Likely_benign                                         20   
         Conflicting_interpretations_of_pathogenicity                329   
         Conflicting_interpretations_of_pathogenicity|other            1   
         Likely_benign                                                58   
         Likely_pathogenic                                           147   
         Likely_pathogenic|other                                       2   
         Pathogenic                                                  368   
         Pathogenic/Likely_pathogenic                                111   
         Pathogenic/Likely_pathogenic|drug_response                    1   
         Pathogenic/Likely_pathogenic|other                            4   
         Pathogenic|drug_response|other                                1   
         Pathogenic|risk_factor                                        1   
         Uncertain_significance                                     1404   
         Uncertain_significance|drug_response                          2   
         drug_response                                                 4   
         not_provided                                                 21   

                                                             DepMap_ID  \
GPD_unit CLNSIG                                                          
LU       .                                                        4850   
         Benign                                                     16   
         Benign/Likely_benign                                        7   
         Conflicting_interpretations_of_pathogenicity              141   
         Likely_benign                                              28   
         Likely_pathogenic                                          49   
         Pathogenic                                                130   
         Pathogenic/Likely_pathogenic                               22   
         Uncertain_significance                                    753   
         not_provided                                                8   
NCU      .                                                        6559   
         Benign                                                     23   
         Benign/Likely_benign                                       87   
         Conflicting_interpretations_of_pathogenicity         

In [72]:
merged_clinvar_df.columns

Index(['Hugo_Symbol', 'DepMap_ID', 'Protein_Change', 'Variant_annotation',
       'sift_pred', 'sift4g_pred', 'lrt_pred', 'mutationtaster_pred',
       'mutationassessor_pred', 'fathmm_pred', 'provean_pred', 'metasvm_pred',
       'm_cap_pred', 'primateai_pred', 'deogen2_pred', 'bayesdel_addaf_pred',
       'bayesdel_noaf_pred', 'clinpred_pred', 'list_s2_pred',
       'fathmm_mkl_coding_pred', 'fathmm_xf_coding_pred', 'gene', 'HGVSp',
       'GPD_unit', 'input', 'SIFT_score', 'SIFT_converted_rankscore',
       'SIFT_pred', 'SIFT4G_score', 'SIFT4G_converted_rankscore',
       'SIFT4G_pred', 'LRT_score', 'LRT_converted_rankscore', 'LRT_pred',
       'MutationTaster_score', 'MutationTaster_converted_rankscore',
       'MutationTaster_pred', 'MutationAssessor_score',
       'MutationAssessor_rankscore', 'MutationAssessor_pred', 'FATHMM_score',
       'FATHMM_converted_rankscore', 'FATHMM_pred', 'PROVEAN_score',
       'PROVEAN_converted_rankscore', 'PROVEAN_pred', 'MetaSVM_pred',
       'M

In [74]:
merged_1[merged_1.mutation == "ABL1 G259G"]

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,...,deogen2_pred,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene,HGVSp,GPD_unit
25692,ABL1,ACH-000878,p.G259G,silent,ABL1 G259G,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,p.G259G,NCU


In [77]:
merged_1["1plusxon17_score"] = 1 + merged_1[CATEGORICAL_COLUMNS].sum(axis=1)/17
merged_1

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,...,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene,HGVSp,GPD_unit,1plusxon17_score
0,TSC2,ACH-000001,p.F297fs,damaging,TSC2 F297fs,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,p.F297fs,PIU,1.000000
1,TP53,ACH-000001,0,other non-conserving,TP53,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,p.,NCU,1.000000
2,CD79B,ACH-000001,p.L21L,silent,CD79B L21L,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,p.L21L,NCU,1.000000
3,SOX9,ACH-000001,p.L3L,silent,SOX9 L3L,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,p.L3L,NCU,1.000000
4,NOTCH3,ACH-000001,p.R1981C,other non-conserving,NOTCH3 R1981C,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,NOTCH3,p.R1981C,PIU,1.647059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62307,BRCA1,ACH-002512,p.H888Y,other non-conserving,BRCA1 H888Y,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BRCA1,p.H888Y,LU,1.176471
62308,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,SMARCA4,p.A400A,NCU,1.764706
62309,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4,p.A400A,NCU,2.000000
62310,SMARCA4,ACH-002512,p.A400A,silent,SMARCA4 A400A,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,SMARCA4,p.A400A,NCU,2.000000


In [84]:
merged_1[merged_1.mutation == "AKT1 E17K"]

,Hugo_Symbol,DepMap_ID,Protein_Change,Variant_annotation,mutation,sift_pred,sift4g_pred,lrt_pred,mutationtaster_pred,mutationassessor_pred,...,bayesdel_addaf_pred,bayesdel_noaf_pred,clinpred_pred,list_s2_pred,fathmm_mkl_coding_pred,fathmm_xf_coding_pred,gene,HGVSp,GPD_unit,1plusxon17_score
11556,AKT1,ACH-000486,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
11557,AKT1,ACH-000486,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
11558,AKT1,ACH-000486,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
48464,AKT1,ACH-001528,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
48465,AKT1,ACH-001528,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
48466,AKT1,ACH-001528,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
52102,AKT1,ACH-001842,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
52103,AKT1,ACH-001842,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
52104,AKT1,ACH-001842,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
52189,AKT1,ACH-001848,p.E17K,other non-conserving,AKT1 E17K,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,AKT1,p.E17K,PIU,1.882353
